# Initial imports

In [27]:
# %pip install inference-sdk 
import os 
import tensorflow as tf
import numpy as np


Set up inference server and expose model as an API endpoint that our backend can access

In [33]:
# %pip install fastapi
# %pip install python-multipart
from fastapi import FastAPI, File, UploadFile
from inference_sdk import InferenceHTTPClient

from dotenv import load_dotenv
from io import BytesIO

# Load environment variables
load_dotenv()
API_KEY = os.getenv("ROBO_API_KEY")

# Initialize Roboflow client
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key=API_KEY
)

app = FastAPI()

@app.get("/")
def read_root():
    print("Root endpoint was hit")  # Debugging log
    return {"message": "HenSense API is running"}


@app.get("/favicon.ico")
async def favicon():
    return {}

# Function to read image file and preprocess it
def read_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3).numpy()
    return image

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    # Read file and convert to numpy array
    image_data = await file.read()
    print("Image data type: ", type(image_data))
    print("Image data: ", image_data)

    # Perform inference
    result = CLIENT.infer(image_data, model_id="healthy-and-sick-chicken-detection-kavqw/18")

    return {"prediction": result}

Start up the server

In [34]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()  # allows Uvicorn to run inside Jupyter

uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [21881]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [21881]


Model fine-tuning using